# COVID-19 cases in Kenya, preprocessing analysis

In [652]:
#load some key libraries

import pandas as pd
import datetime as dt
from datetime import date
import calendar
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [653]:
#select_important_columns
#covid_data_original = pd.read_excel("Covid-19_cleaned_07-02-2022.xls", engine="xlrd")
covid_data_original = pd.read_table("Covid-19_cleaned_07-02-2022.txt")

covid_data_original["county_where_the_case_was_diagonised"] =  covid_data_original["county_where_the_case_was_diagonised"].\
                                                            apply(lambda x : str(x).title())
covid_data_original["county_where_the_case_was_diagonised"] = covid_data_original["county_where_the_case_was_diagonised"].\
                    replace({"Murang'A": "Murang'a", "Kakamega ": "Kakamega", "Nakuru ": "Nakuru", "Kajiado ":"Kajiado"})


covid_data_original.head()

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_7815/2193926576.py:3: DtypeWarning:

Columns (2,3,15,16,17,25,26,28,29,35,36,37,38,39,40,42,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.



,unique_id,case_id,contact_of_new,age_years,sex,occupation_sector_job_title_employer_,county_of_residence,county_where_the_case_was_diagonised,nationality,sub_county,...,co_morbidity,management,status_need,date_of_announcement_by_cs,type_of_testing,graph_date,status,total_deaths,history_of_contact_new,symptomatic_yes_no_new
0,Case-01,Case 1,NaN,27.0,F,/Unemployed,Kajiado,Kajiado,Kenyan,Kajiado North,...,NaN,NaN,NaN,13-Mar-20,PCR,NaN,#ERROR!,1241,NaN,YES
1,Case-02,Case 2,NaN,48.0,F,/Unknown,Nairobi,Nairobi,Kenyan,Starehe,...,NaN,NaN,NaN,NaN,PCR,NaN,#ERROR!,NaN,NaN,YES
2,Case-03,Case 3,Case-01,27.0,M,/Unknown,Nairobi,Nairobi,Kenyan,Kajiado North,...,NaN,NaN,NaN,NaN,PCR,NaN,#ERROR!,NaN,YES,YES
3,Case-04,Case 4,NaN,31.0,F,/Unknown,Nairobi,Nairobi,Kenyan,Langata,...,NaN,NaN,NaN,NaN,PCR,NaN,#ERROR!,NaN,NaN,YES
4,Case-05,Case 5,NaN,42.0,F,/Unknown,Nairobi,Nairobi,Kenyan,Westlands,...,NaN,NaN,NaN,NaN,PCR,NaN,#ERROR!,NaN,NaN,YES


Visualizing lab_results column returns presence of wrong labels and names of hospitals. Creating a new colum fpor confirmed positives. This will determine the total cases

In [655]:
covid_data = covid_data_original[["unique_id","age_years", "sex","county_of_residence","county_where_the_case_was_diagonised",
                                    "status_need","status","lab_results","date_of_lab_confirmation","outcome_death_Discharge_still_in_hospital_"]]

covid_data.tail()

,unique_id,age_years,sex,county_of_residence,county_where_the_case_was_diagonised,status_need,status,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_
322122,Case-0322092,21.0,F,Siaya,Siaya,NaN,NaN,NaN,05/02/2022,NaN
322123,Case-0322093,2.0,M,Siaya,Siaya,NaN,NaN,NaN,05/02/2022,NaN
322124,Case-0322094,70.0,M,Tharaka Nithi,Tharaka Nithi,NaN,NaN,NaN,05/02/2022,NaN
322125,Case-0322095,15.0,F,Uasin Gishu,Uasin Gishu,NaN,NaN,NaN,05/02/2022,NaN
322126,Case-0322096,43.0,M,Uasin Gishu,Uasin Gishu,NaN,NaN,NaN,05/02/2022,NaN


In [656]:
#convert the date column to true dates
covid_data["date_of_lab_confirmation"] = pd.to_datetime(covid_data["date_of_lab_confirmation"], format="%d/%m/%Y", errors="coerce")

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_7815/2772849741.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Plots based on lab_results and outcome from hospitals

In [657]:
covid_data_view = covid_data[[ "sex","age_years", "lab_results", "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" , "county_where_the_case_was_diagonised"]]

#there exists some cases with missing date of lab confirmation. Dropping them
covid_data_view=covid_data_view.dropna(subset=["date_of_lab_confirmation"])

covid_data_view.tail()

,sex,age_years,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_,county_where_the_case_was_diagonised
322122,F,21.0,NaN,2022-02-05,NaN,Siaya
322123,M,2.0,NaN,2022-02-05,NaN,Siaya
322124,M,70.0,NaN,2022-02-05,NaN,Tharaka Nithi
322125,F,15.0,NaN,2022-02-05,NaN,Uasin Gishu
322126,M,43.0,NaN,2022-02-05,NaN,Uasin Gishu


In [658]:
covid_data_view["Year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("Y")
covid_data_view["Month"] =  covid_data_view["date_of_lab_confirmation"].dt.month
covid_data_view["Week"] = covid_data_view["date_of_lab_confirmation"].dt.week
covid_data_view["Month"] =  covid_data_view["Month"].apply(lambda x: calendar.month_abbr[x])
covid_data_view["Month_year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("M")
covid_data_view["year_month"] = covid_data_view["Month"] + "-" +  covid_data_view["Year"].astype(str) 
covid_data_view.head()

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_7815/3999872376.py:3: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



,sex,age_years,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_,county_where_the_case_was_diagonised,Year,Month,Week,Month_year,year_month
0,F,27.0,Positive,2020-03-12,Discharge,Kajiado,2020,Mar,11,2020-03,Mar-2020
1,F,48.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
2,M,27.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
3,F,31.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020
4,F,42.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020


In [659]:
Monthly_cases = covid_data_view[["Month_year", "year_month", "outcome_death_Discharge_still_in_hospital_" ,"lab_results"]].\
    groupby(["Month_year", "year_month"])["Month_year"].count().to_frame(name = "COVID19_Cases").reset_index()
Monthly_cases.rename(columns = {"COVID19_Cases": "Reported_cases"}, inplace=True)

death_cases = covid_data_view[covid_data_view["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
monthly_death_cases = death_cases[["outcome_death_Discharge_still_in_hospital_","Month_year", "year_month"]].\
    groupby(["Month_year", "year_month"]).count().reset_index()
monthly_death_cases.rename(columns = {"outcome_death_Discharge_still_in_hospital_" : "Death_cases"}, inplace=True)

covid_monthly_data = pd.merge(Monthly_cases, monthly_death_cases, on = "year_month", how = "outer")
covid_monthly_data =  covid_monthly_data.drop("Month_year_x", axis=1)
covid_monthly_data =  covid_monthly_data.rename(columns={"Month_year_y" : "Month_year"})
covid_monthly_data["Month_year"]= covid_monthly_data["Month_year"].dt.strftime("%Y-%m")

covid_monthly_data.to_csv("covid_monthly_data.csv")


In [660]:
Monthly_cases = pd.read_csv("covid_monthly_data.csv")

Monthly_cases.head()

,Unnamed: 0,year_month,Reported_cases,Month_year,Death_cases
0,0,Mar-2020,84,2020-03,5
1,1,Apr-2020,350,2020-04,20
2,2,May-2020,1701,2020-05,66
3,3,Jun-2020,4719,2020-06,134
4,4,Jul-2020,14480,2020-07,347


Daily Cases

In [661]:
#summary for daily reported dcases
daily_cases =  covid_data_view[["date_of_lab_confirmation", "lab_results"]].groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].\
    count().to_frame(name = "Reported_Cases").reset_index()
daily_cases.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)
#daily_cases.tail()

#summary for dail deaths
daily_deaths = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
daily_deaths = daily_deaths[[ "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" ]].\
    groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].count().to_frame(name="death_cases").reset_index()
daily_deaths.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)

#summary for dicharged
daily_discharges = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Discharge"]
daily_discharges = daily_discharges[[ "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" ]].\
    groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].count().to_frame(name="Reported_discharges").reset_index()
daily_discharges.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)

covid_daily_data = pd.merge(pd.merge(daily_cases, daily_deaths, how = "outer"), daily_discharges, how="outer")

covid_daily_data["death_cases"] = covid_daily_data["death_cases"].fillna(0)
covid_daily_data["Reported_discharges"] = covid_daily_data["Reported_discharges"].fillna(0)

covid_daily_data = covid_daily_data.astype({"Reported_Cases":int, "death_cases": int, "Reported_discharges":int })
covid_daily_data["Cum_Cases"] = covid_daily_data["Reported_Cases"].cumsum()
covid_daily_data["Cum_Deaths"] = covid_daily_data["death_cases"].cumsum()
covid_daily_data.to_csv("covid_daily_data.csv")
covid_daily_data.head()

covid_daily_data.to_csv("covid_daily_data.csv")
covid_daily_data.head()

,Date,Reported_Cases,death_cases,Reported_discharges,Cum_Cases,Cum_Deaths
0,2020-03-12,1,0,1,1,0
1,2020-03-14,2,0,2,3,0
2,2020-03-17,4,0,4,7,0
3,2020-03-19,3,0,3,10,0
4,2020-03-20,1,0,1,11,0


In [663]:
from plotly.graph_objects import Layout
layout = Layout(plot_bgcolor="rgba(0,0,0,0)")
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter( x = covid_daily_data["Date"],  y = covid_daily_data["Reported_Cases"],mode='none',
                            fill = 'tozeroy', marker = dict(color = "#A3A3A3")))
fig.update_layout(title = "COVID-19 Monthly Cases", font_color = "darkcyan")
fig.update_yaxes(title =  "Counts", showline=True, linewidth = 0.2, linecolor = "gray", gridcolor = "gainsboro")
fig.update_xaxes(title = "Period", showgrid=False)
fig.show()

Weekly Cases

In [664]:
#export data

covid_data.to_csv("covid_data_processed.csv")

In [665]:
#filter data for those who died
dead = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
daily_deaths = dead[[ "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" ]].\
    groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].count().to_frame(name="death_cases").reset_index()
daily_deaths.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)

daily_deaths.to_csv("death_cases.csv")

from plotly.graph_objects import Layout
layout = Layout(plot_bgcolor="rgba(0,0,0,0)")
fig5 = go.Figure(layout=layout)
fig5.add_trace(go.Scatter( x = daily_deaths.Date,  y = daily_deaths.death_cases,mode='none',
                            fill = 'tozeroy', marker = dict(color = "#A3A3A3")))
fig5.update_layout(title = "COVID-19 Daily Deaths", font_color = "darkcyan")
fig5.update_yaxes(title =  "Deaths Cases", showline=True, linewidth = 0.2, linecolor = "gray", gridcolor = "gainsboro")
fig5.update_xaxes(title = "Period", showgrid=False)
#fig.show()

## Visualization Based on County where case was diagnoised


In [669]:
#covid_data.rename(columns = {"county_where_the_case_was_diagonised" :"County"}, inplace=True)
county_death = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
#county_death.rename(columns = {"county_where_the_case_was_diagonised" :"County"}, inplace=True)

county_death_counts = county_death[["county_where_the_case_was_diagonised", "outcome_death_Discharge_still_in_hospital_"]].\
                        groupby("county_where_the_case_was_diagonised").count().reset_index()
                        
county_death_counts.rename(columns={"outcome_death_Discharge_still_in_hospital_": "Death_cases"}, inplace=True)
#county_death_counts

discharged = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Discharge"]
discharged = discharged[["county_where_the_case_was_diagonised", "outcome_death_Discharge_still_in_hospital_"]].\
                    groupby("county_where_the_case_was_diagonised").count().reset_index()
discharged.rename(columns={"outcome_death_Discharge_still_in_hospital_": "Discharged"}, inplace=True)
#discharged

#covid_data["county_of_residence"] = covid_data["county_where_the_case_was_diagonised"].str.strip().str.title()#.unique() #Needs clean-up for naming consistency
county_prevalence = covid_data.groupby("county_where_the_case_was_diagonised")["county_where_the_case_was_diagonised"].count().\
                to_frame(name = "cases").reset_index()
county_prevalence = county_prevalence.sort_values("cases", ascending=False)
#county_prevalence

county_cases = pd.merge(pd.merge(county_prevalence, county_death_counts, on="county_where_the_case_was_diagonised"),\
                    discharged,how="left", on="county_where_the_case_was_diagonised")

county_cases["Discharged"] = county_cases["Discharged"].fillna(0)
county_cases = county_cases.astype({"cases":int, "Death_cases": int, "Discharged":int })

county_cases.rename(columns={"county_where_the_case_was_diagonised": "County"}, inplace=True)

#include sequenced, vaccination, status


county_cases.to_csv("cases_per_county.csv") #check merged with sequence information

county_cases.head()

,County,cases,Death_cases,Discharged
0,Nairobi,130228,1542,3561
1,Kiambu,20127,545,263
2,Mombasa,17887,337,1530
3,Nakuru,16947,531,66
4,Uasin Gishu,10507,241,111


## Vaccination information

In [670]:
#data from ministry of health 

County = ["Baringo","Bomet","Bungoma","Busia","Elgeyo Marakwet","Embu","Garissa","Homa Bay","Isiolo","Kajiado","Kakamega","Kericho","Kiambu",\
    "Kilifi","Kirinyaga","Kisii","Kisumu","Kitui","Kwale","Laikipia","Lamu","Machakos","Makueni","Mandera","Marsabit","Meru","Migori","Mombasa",\
            "Murang'a","Nairobi","Nakuru","Nandi","Narok","Nyamira","Nyandarua","Nyeri","Samburu","Siaya","Taita Taveta","Tana River","Tharaka Nithi",\
                "Trans Nzoia","Turkana","Uasin Gishu","Vihiga","Wajir","West Pokot"]
Population_vaccinated = [84904,76594,279522,161039,66826,133100,54335,233810,17548,231369,434677,166407,594614,136859,157439,258899,293662,138916,\
                                78222,122869,19682,323810,192008,35563,22963,282518,220721,275349,253870,1597784,518376,132910,84855,141630,124173,\
                                    273192,31802,237793,82965,16606,85121,191032,102890,242715,113992,37371,41559]
                    
Proportion_vaccinated = [25.1,16.4,33.1,34.6,27.4,33.6,13.6,38.4,13.2,34.9,44.2,32.7,36.6,17.4,38.1,37.1,44.8,22.2,17.3,40.1,23.6,35.8,32.9,10.4,10.5,\
                        29.8,39.6,34.2,37.6,52.3,40.7,27,15.5,41.5,33,53.6,21.8,44.4,38.9,11,34.5,36.9,21.9,35,34.6,11.1,14.6]

#pd.DataFrame(zip(County,Population_vaccinated,Proportion_vaccinated))
vaccination_per_county = pd.DataFrame(zip(County, Population_vaccinated, Proportion_vaccinated), columns = ["County","Population_vaccinated", "Proportion_vaccinated" ])

#vaccination_per_county.to_csv("county_vaccination.csv")
vaccination_per_county.sort_values("Proportion_vaccinated", ascending=True).to_csv("county_vaccination.csv")


# creating regions from county

In [671]:
county_cases.head()

,County,cases,Death_cases,Discharged
0,Nairobi,130228,1542,3561
1,Kiambu,20127,545,263
2,Mombasa,17887,337,1530
3,Nakuru,16947,531,66
4,Uasin Gishu,10507,241,111


In [672]:

regions = dict(
    {"Coast" : ["Mombasa", "Kwale", "Kilifi", "Tana River", "Taita Taveta", "Lamu"],
    "North Eastern" : [  "Garissa", "Wajir", "Mandera"],
    "Eastern" : ["Marsabit", "Isiolo", "Meru", "Tharaka Nithi", "Embu", "Kitui", "Machakos", "Makueni"],
    "Central" : [ "Nyandarua", "Murang'a", "Kirinyaga", "Nyeri", "Kiambu"],
    "Rift Valley" : ["Nakuru", "Narok", "Kajiado", "Kericho", "Bomet", "Turkana", "West Pokot", "Samburu","Trans Nzoia", "Uasin Gishu", "Elgeyo Marakwet", "Nandi", "Baringo", "Laikipia"],
    "Western": ["Kakamega", "Vihiga", "Bungoma", "Busia"],
    "Nyanza": [ "Siaya", "Kisumu", "Homa Bay", "Migori", "Kisii", "Nyamira"],
    "Nairobi": ["Nairobi"]
    }) 

#print(regions.values())
province = []
for name in county_cases["County"]:
    for (k,v) in regions.items():
        for i in v:
            if i == name:
                province.append(k)
        

county_cases["province"] =  province

county_cases.head()

,County,cases,Death_cases,Discharged,province
0,Nairobi,130228,1542,3561,Nairobi
1,Kiambu,20127,545,263,Central
2,Mombasa,17887,337,1530,Coast
3,Nakuru,16947,531,66,Rift Valley
4,Uasin Gishu,10507,241,111,Rift Valley


In [673]:
import geopandas as gpd
#import geoplot as gplt
#import geoplot.crs as gcrs
#import imageio
import matplotlib.pyplot as plt
#import mpclassift as pc


province_cases = county_cases.groupby(["province"])["cases","Death_cases", "Discharged"].sum().reset_index()

kenya_province = gpd.read_file("Admin2/KEN_admin2_2002_DEPHA.shp")
kenya_province["PROVINCE"] =  kenya_province["PROVINCE"].str.replace("\r\n", "").str.title()

province_cases = pd.merge(kenya_province, province_cases, how="left", left_on=["PROVINCE"], right_on=["province"])

province_cases.to_csv("provice_level_cases.csv")


province_cases

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_7815/3202668277.py:9: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,PROVINCE,Prov_Pcode,geometry,province,cases,Death_cases,Discharged
0,Central,KE2,"POLYGON ((37.31979 -0.75808, 37.31935 -0.75773...",Central,35648,1085,284
1,Coast,KE3,"MULTIPOLYGON (((39.06581 -3.02189, 39.05392 -2...",Coast,29419,439,1684
2,Eastern,KE4,"POLYGON ((38.95712 2.08819, 38.95787 2.08736, ...",Eastern,25266,634,193
3,Nairobi,KE1,"POLYGON ((36.68167 -1.29539, 36.68264 -1.29358...",Nairobi,130228,1542,3561
4,North Eastern,KE5,"POLYGON ((40.97417 2.15810, 40.97480 2.15811, ...",North Eastern,3381,138,67
5,Nyanza,KE6,"MULTIPOLYGON (((35.04765 -0.70610, 35.05234 -0...",Nyanza,25418,488,182
6,Rift Valley,KE7,"POLYGON ((36.60691 2.39510, 36.60953 2.39583, ...",Rift Valley,57786,1089,545
7,Western,KE8,"MULTIPOLYGON (((35.15522 0.68490, 35.15526 0.6...",Western,14947,206,442


In [674]:
province_cases_sort = province_cases[["province", "cases"]]
province_cases_sort = province_cases_sort.sort_values("cases", ascending = True)
fig = px.histogram(province_cases_sort,
               y="province",x="cases",
               orientation="h",title="Region Level Cases",
               height=500, width=900
               )
fig.update_layout(uniformtext_minsize = 3, font_color = "darkcyan", legend = dict(bgcolor = "white"), bargap =0.05)
fig.update_yaxes(title =  "Counts", showline=True, linewidth = 0.2, linecolor = "gray", gridcolor = "gainsboro")
fig.update_xaxes(title = "Period", showgrid=False)

#fig.show()

#fig.write_image("county_cases_image.png")

## Visualizing death cases

Prepare dataset to be used for county level comparison - time series

In [675]:
county_daily_data = covid_data_view[covid_data_view["lab_results"] == "Positive"]

county_daily_data = county_daily_data.rename(columns = { "outcome_death_Discharge_still_in_hospital_": "outcome", 
                            "county_where_the_case_was_diagonised" : "county"})

#county_daily_data.to_csv("county_daily_data.csv")
county_daily_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319295 entries, 0 to 322073
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   sex                       319293 non-null  object        
 1   age_years                 318244 non-null  object        
 2   lab_results               319295 non-null  object        
 3   date_of_lab_confirmation  319295 non-null  datetime64[ns]
 4   outcome                   12417 non-null   object        
 5   county                    319295 non-null  object        
 6   Year                      319295 non-null  period[A-DEC] 
 7   Month                     319295 non-null  object        
 8   Week                      319295 non-null  int64         
 9   Month_year                319295 non-null  period[M]     
 10  year_month                319295 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7), period[A-DEC](1), peri

## Age based analysis with gender

In [676]:
age_gender = county_daily_data.reset_index()[["sex", "age_years", "lab_results"]]
age_gender =  age_gender[(age_gender["age_years"].notna()) & (age_gender["age_years"] != "M")]  #take rows that are not na
age_gender["age_years"] =  age_gender["age_years"].astype(int)
age_gender = age_gender[(age_gender["age_years"] > 0) & age_gender["age_years"] < 100]
bins  = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,100 ]
labels = ["0-5", "6-10", "11-15","16-20","21-25", "25-30", "31-35", "36-40", "41-45","46-50","51-55","56-60", "61-65","66-70","71-75",
            "76-80","81-85","86-90","90+"]
age_gender["age_groups"] = pd.cut(age_gender.age_years, bins, labels=labels, include_lowest=True)

age_gender["sex"] = age_gender["sex"].replace({"m":"M", "f":"F"})
age_gender = age_gender.rename(columns={"sex":"Gender"})
age_gender = age_gender[["Gender","age_years","age_groups"]].groupby(["age_groups","Gender"]).count().reset_index()
age_gender = pd.pivot_table(age_gender, values="age_years", index = ["age_groups"],columns = ["Gender"])
age_gender = age_gender.rename(columns = {"F":"Female","M":"Male"})
age_gender.to_csv("age_gender_cases_data.csv")


### Age gender death cases

In [678]:

age_gender_death = county_daily_data[county_daily_data["outcome"] == "Dead"]

age_gender_death= age_gender_death.reset_index()[["sex", "age_years","outcome"]]
age_gender_death =  age_gender_death[(age_gender_death["age_years"].notna()) & (age_gender_death["age_years"] != "M")]
age_gender_death["age_years"] =  age_gender_death["age_years"].astype(int)
age_gender_death = age_gender_death[(age_gender_death["age_years"] > 0) & age_gender_death["age_years"] < 100]

age_gender_death["age_groups"] = pd.cut(age_gender_death.age_years, bins, labels=labels, include_lowest=True)
age_gender_death["sex"] = age_gender_death["sex"].replace({"m":"M", "f":"F"})
age_gender_death = age_gender_death.rename(columns={"sex":"Gender"})
age_gender_death = age_gender_death[["Gender","age_years","age_groups"]].groupby(["age_groups","Gender"]).count().reset_index()
age_gender_death = pd.pivot_table(age_gender_death, values="age_years", index = ["age_groups"],columns = ["Gender"])
age_gender_death = age_gender_death.rename(columns = {"F":"Female","M":"Male"})

age_gender_death.to_csv("age_gender_death_cases_data.csv")
age_gender_death.head()

Gender,Female,Male
age_groups,,
0-5,27,49
6-10,6,10
11-15,7,16
16-20,17,20
21-25,30,34


In [679]:
county_daily_data_selected = county_daily_data[county_daily_data["county"].isin(["Nairobi", "Mombasa"])]
county_daily_data_selected = county_daily_data_selected[[ "county", "lab_results", "date_of_lab_confirmation"]].\
                                groupby(["county", "date_of_lab_confirmation"]).count().reset_index()

fig = px.line(county_daily_data_selected, x="date_of_lab_confirmation" , y = "lab_results" , color = "county")
fig.update_traces(mode = 'lines')
fig.update_layout()
fig.show()

In [680]:
county_daily_data["date_of_lab_confirmation"] = pd.to_datetime(county_daily_data["date_of_lab_confirmation"])
county_daily_data["Date"] = county_daily_data["date_of_lab_confirmation"]#.dt.date
county_daily_data.set_index("Date", inplace = True)


## Seroprecalence Data preprocessing

In [681]:
sero_data = pd.read_excel("KWTRP_serosurveillance_data_Dashboard_09Sep2022.xlsx") #requires openpyxl

In [682]:
sero_data["Period"] = sero_data[["Month(s)", "Year"]].astype(str).agg(" ".join, axis=1)

In [683]:
blood_donors = sero_data[sero_data["Population"] == "Blood donors"][["Population","Age in years"]]
blood_donors = blood_donors.groupby("Age in years").count().reset_index()
blood_donors = blood_donors.rename(columns={"Population":"Frequency"})
fig = go.Figure()
fig.add_trace(go.Bar(x = blood_donors["Age in years"], y = blood_donors["Frequency"]))
fig.update_layout(uniformtext_minsize = 3, font_color = "#000000", bargap =0.2,font_size=10,autosize=False, width = 600, height=800)
fig.show()


In [685]:
Health_workers = sero_data[sero_data["Population"] == "Health workers"][["Population","Age in years"]]
Health_workers = Health_workers.groupby("Age in years").count().reset_index()
Health_workers
Truck_crews = sero_data[sero_data["Population"] == "Trucking crews"][["Population","Age in years"]]
Truck_crews = Truck_crews.groupby("Age in years").count().reset_index()
Truck_crews

,Age in years,Population
0,18 - 30,1
1,31 - 40,1
2,41 - 50,1
3,51 - 60,1
4,>60,1
5,≥18,6


In [687]:
anc_attendees = sero_data[sero_data["Population"] == "ANC attendees"][["Population","Age in years"]]
anc_attendees = anc_attendees.groupby("Age in years").count().reset_index()
anc_attendees

,Age in years,Population
0,17 - 45,1
1,UNK,3


In [688]:
HDSS = sero_data[sero_data["Population"] == "HDSS residents"][["Population","Age in years"]]
HDSS = HDSS.groupby("Age in years").count().reset_index()
HDSS = HDSS.rename(columns={"Population":"Frequency"})
HDSS["Values"] = [3,4,5,6,7,2,1,8]
HDSS = HDSS.sort_values(by="Values")
fig = go.Figure()
fig.add_trace(go.Bar(x = HDSS["Age in years"], y = HDSS["Frequency"]))
fig.update_layout(uniformtext_minsize = 3, font_color = "#000000", bargap =0.2,font_size=10,autosize=False, width = 600, height=800)
fig.show()

In [689]:

from plotly.subplots import make_subplots

fig = make_subplots(rows = 3,cols=3)
fig.add_trace(go.Bar(x = blood_donors["Age in years"], y = blood_donors["Frequency"],name = "Blood donors"), row=1,col=1)
fig.add_trace(go.Bar(x = HDSS["Age in years"], y = HDSS["Frequency"], name = "HDSS"), row=1,col=2)
fig.add_trace(go.Bar(x = Health_workers["Age in years"], y = Health_workers["Population"],name = "Health workers"), row=2,col=1)
fig.add_trace(go.Bar(x = Truck_crews["Age in years"], y = Truck_crews["Population"],name = "Truck crews"), row=2,col=2)
fig.add_trace(go.Bar(x = anc_attendees["Age in years"], y = anc_attendees["Population"],name = "ANC Attendees"), row=2,col=3)
fig.update_yaxes( showline=True, linewidth = 0.2, linecolor = "gray", gridcolor = "gainsboro")
fig.update_xaxes( showline=True, linewidth = 0.2,linecolor = "gray")
fig.update_traces(width = 0.5)
fig.update_layout(height=700, width=900, title = "Sero-study population categories", plot_bgcolor = "#FFFAFA", paper_bgcolor = "#FFFAFA")

fig.show()

In [690]:
gender = sero_data["Sex"].value_counts().to_frame()
gender["%"] = round(gender["Sex"]/gender["Sex"].sum() * 100, 2 )
fig = px.pie(gender, values = "%", names = gender.index, hole = .4)
#gender
fig.update_layout(height = 400, width = 500)
fig.show()

In [691]:
population = sero_data["Population"].value_counts().to_frame()
population["%"] = round(population["Population"]/population["Population"].sum() * 100, 2)
fig = px.pie(population, values = "%",names=population.index, hole = .4)
fig.update_layout(height = 400, width = 500)



In [692]:
sero_data.head()

,Population,Region,Age in years,Sex,Month(s),Year,Anti-spike IgG seroprevalence,LB 95% margin of error,UB 95% margin of error,Source,Period
0,Blood donors,National,15 - 64,All,Apr - Jun,2020,4.3,2.9,4.8,doi: 10.1126/science.abe1916,Apr - Jun 2020
1,Blood donors,National,15 - 64,Male,Apr - Jun,2020,3.6,1.9,5.8,doi: 10.1126/science.abe1916,Apr - Jun 2020
2,Blood donors,National,15 - 64,Female,Apr - Jun,2020,4.8,3.5,6.4,doi: 10.1126/science.abe1916,Apr - Jun 2020
3,Blood donors,National,15 - 24,All,Apr - Jun,2020,4.4,2.7,6.4,doi: 10.1126/science.abe1916,Apr - Jun 2020
4,Blood donors,National,25 - 34,All,Apr - Jun,2020,4.2,2.3,6.0,doi: 10.1126/science.abe1916,Apr - Jun 2020


### Distribution by Period

In [693]:
period = sero_data[["Population","Period"]].groupby(["Period","Population"])["Period"].count().to_frame()
period.index.names = ["Periods","Population"]
period = period.reset_index()
period = period.rename(columns = {"Periods" :"Period", "Period":"Frequency"})

period

,Period,Population,Frequency
0,Apr - Jun 2020,Blood donors,16
1,Aug - Sep 2020,Blood donors,16
2,Dec - Apr 2021,HDSS residents,10
3,Feb - May 2021,HDSS residents,10
4,Feb - May 2022,HDSS residents,10
5,Jan - Mar 2021,Blood donors,16
6,Jan - May 2021,HDSS residents,10
7,Jul - Aug 2020,ANC attendees,1
8,Jul - Dec 2020,Health workers,11
9,Jun - Aug 2020,Blood donors,16


In [694]:
fig = px.bar(period, x = "Period", y = "Frequency", barmode="group", color = "Population")
fig.update_layout(height = 400, width = 1000, bargap = 0.005)

In [695]:
region = sero_data["Region"].value_counts().to_frame()
region["%"] = round(region["Region"]/region["Region"].sum() * 100, 2)
fig = px.pie(region, values = "%",names=region.index, hole = .4)
fig.update_layout(height = 500, width = 700)

In [696]:
sero_data.head()

,Population,Region,Age in years,Sex,Month(s),Year,Anti-spike IgG seroprevalence,LB 95% margin of error,UB 95% margin of error,Source,Period
0,Blood donors,National,15 - 64,All,Apr - Jun,2020,4.3,2.9,4.8,doi: 10.1126/science.abe1916,Apr - Jun 2020
1,Blood donors,National,15 - 64,Male,Apr - Jun,2020,3.6,1.9,5.8,doi: 10.1126/science.abe1916,Apr - Jun 2020
2,Blood donors,National,15 - 64,Female,Apr - Jun,2020,4.8,3.5,6.4,doi: 10.1126/science.abe1916,Apr - Jun 2020
3,Blood donors,National,15 - 24,All,Apr - Jun,2020,4.4,2.7,6.4,doi: 10.1126/science.abe1916,Apr - Jun 2020
4,Blood donors,National,25 - 34,All,Apr - Jun,2020,4.2,2.3,6.0,doi: 10.1126/science.abe1916,Apr - Jun 2020


In [740]:
BD_sero = sero_data[sero_data["Population"] == "Blood donors"]


In [699]:
sero_plot = px.box(sero_data, x = "Population", y = "Anti-spike IgG seroprevalence",color="Population", points="all")#boxmode="overlay",facet_col = "Population",
sero_plot.show()

## Genomics Data Preprocessing
Sequence metadata were downloaded from GISAID Databases and merged together

In [700]:
import glob

path  = "/Users/jmwanga/OneDrive - Kemri Wellcome Trust/Visualization/"
all_files = glob.glob(os.path.join(path, "*.tsv"))
list = []
for file in all_files:
    df = pd.read_table(file)
    list.append(df)
seq_metadata = pd.concat(list)
seq_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11156 entries, 0 to 1568
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Virus name                       11156 non-null  object 
 1   Accession ID                     11156 non-null  object 
 2   Collection date                  11156 non-null  object 
 3   Location                         11156 non-null  object 
 4   Host                             11156 non-null  object 
 5   Additional location information  843 non-null    object 
 6   Sampling strategy                2578 non-null   object 
 7   Gender                           11145 non-null  object 
 8   Patient age                      11129 non-null  object 
 9   Patient status                   11156 non-null  object 
 10  Last vaccinated                  0 non-null      float64
 11  Passage                          11156 non-null  object 
 12  Specimen           

In [701]:
seq_metadata["Region"] = seq_metadata["Location"].str.split("/",2,expand=True)[2] #select the third colum representing counties

seq_metadata["Region"] = seq_metadata.Region.fillna("unknown")
seq_metadata["Region"] = seq_metadata["Region"].str.replace(" ","")

seq_metadata[seq_metadata.Region == "unknown"]

unknown_regions = []
for name in seq_metadata["Region"]:
    if name not in County:
        unknown_regions.append(name)

unknown_regions = set(unknown_regions) #get unique values

In [702]:

for name in seq_metadata["Region"]:
    if name.startswith("Bomet"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Bomet"})
    elif name.startswith("Dadaab"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Garissa"})
    elif name.startswith("Homa"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Homa Bay"})
    elif name.startswith("Kerugoya"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Kirinyaga"})
    elif name.startswith(("Kuria")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Migori"})
    elif name.startswith(("Machakos","MACHA")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Machakos"})
    elif name.startswith("Makindu"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Makueni"})
    elif name.startswith(("Malaba","Buisa")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Busia"})
    elif name.startswith("Meru"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Meru"})
    elif name.startswith("Moyale"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Marsabit"})
    elif name.startswith(("EMT","Elgeyo")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Elgeyo Marakwet"})
    elif name.startswith("Trans"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Trans Nzoia"})
    elif name.startswith("Keri"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Kericho"})
    elif name.startswith(("Westp","Pokot")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"West Pokot"})
    elif name.startswith(("Uasin","Kapseret","Eldoret")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Uasin Gishu"})
    elif name.startswith("Tharaka"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Tharaka Nithi"})
    elif name.startswith("Tana"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Tana River"})
    elif name.startswith("Taita"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Taita Taveta"})
    elif name.startswith("Nairob"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Nairobi"})
    elif name.startswith(("Thika","KIAMBU")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Kiambu"})
    elif name.startswith(("Wakiso","NorthKivu","Mukono","Kyadondo","Kampala")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"unknown"})
    elif name.startswith(("Nyando","Kisumu")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Kisumu"})
    elif name.startswith(("Nanyuki","Laiki")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Laikipia"})
    elif name.startswith("Naivasha"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Nakuru"})
    elif name.startswith("Muran"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Murang'a"})


In [724]:
seq_metadata.head()

,Virus name,Accession ID,Collection date,Location,Host,Additional location information,Sampling strategy,Gender,Patient age,Patient status,Last vaccinated,Passage,Specimen,Additional host information,Lineage,Clade,AA Substitutions,Region
0,hCoV-19/Kenya/C117586/2022,EPI_ISL_13521106,2022-06-14,Africa / Kenya / Kilifi,Human,NaN,NaN,Female,15,unknown,NaN,Original,NaN,NaN,BA.5.2.1,GRA,"(NSP5_P132H,Spike_H69del,NSP3_G489S,Spike_L24d...",Kilifi
1,hCoV-19/Kenya/C117773/2022,EPI_ISL_13521097,2022-06-17,Africa / Kenya / Kilifi,Human,NaN,NaN,Male,1,unknown,NaN,Original,NaN,NaN,BA.4.1,GRA,"(NSP5_P132H,Spike_H69del,Spike_V3G,NSP3_G489S,...",Kilifi
2,hCoV-19/Kenya/C176297/2022,EPI_ISL_13521099,2022-06-13,Africa / Kenya / Kilifi,Human,NaN,NaN,Male,1,unknown,NaN,Original,NaN,NaN,BA.2.36,GRA,"(NSP5_P132H,NSP3_G489S,Spike_L24del,NSP4_T327I...",Kilifi
3,hCoV-19/Kenya/SS9067/2022,EPI_ISL_13521109,2022-06-10,Africa / Kenya / Kiambu,Human,NaN,NaN,Male,16,unknown,NaN,Original,NaN,NaN,BA.5.2.1,GRA,"(NSP5_P132H,Spike_H69del,NSP3_G489S,Spike_L24d...",Kiambu
4,hCoV-19/Kenya/C117680/2022,EPI_ISL_13521093,2022-06-14,Africa / Kenya / Kilifi,Human,NaN,NaN,Male,0,unknown,NaN,Original,NaN,NaN,BA.5.2.1,GRA,"(NSP5_P132H,Spike_H69del,NSP3_G489S,Spike_L24d...",Kilifi


In [ ]:

# covid_data_view["Year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("Y")
# covid_data_view["Month"] =  covid_data_view["date_of_lab_confirmation"].dt.month
# covid_data_view["Week"] = covid_data_view["date_of_lab_confirmation"].dt.week
# covid_data_view["Month"] =  covid_data_view["Month"].apply(lambda x: calendar.month_abbr[x])
# covid_data_view["Month_year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("M")
# covid_data_view["year_month"] = covid_data_view["Month"] + "-" +  covid_data_view["Year"].astype(str) 

In [758]:
seq_metadata["collection_date"] =  pd.to_datetime(seq_metadata["Collection date"], format="%Y-%m-%d")


In [766]:
daily_seq_data = seq_metadata[["collection_date", "Accession ID"]].groupby("collection_date").count().reset_index()
daily_seq_data = daily_seq_data.rename(columns={"Accession ID":"sequences"})
daily_seq_data["cum_sequences"] = daily_seq_data["sequences"].cumsum()

daily_seq_data.head()

,collection_date,sequences,cum_sequences
0,2020-03-01,9,9
1,2020-03-06,1,10
2,2020-03-14,1,11
3,2020-03-17,2,13
4,2020-03-20,1,14


In [730]:
seq_by_region = seq_metadata[["Location","Region"]].groupby("Region").count().reset_index()
seq_by_region= seq_by_region.rename(columns={"Region":"County", "Location":"sequences"})
seq_by_region


,County,sequences
0,Baringo,16
1,Bomet,18
2,Bungoma,186
3,Busia,124
4,Elgeyo Marakwet,7
5,Embu,2
6,Garissa,60
7,Homa Bay,64
8,Isiolo,2
9,Kajiado,44


In [767]:
seq_by_region.to_csv("sequence_metadata_region.tsv", sep="\t")
daily_seq_data.to_csv("sequence_metadata.tsv", sep="\t")


## Trends in sample sequencing

## Location

In [707]:
seq_metadata[["Virus name", "Collection date"]].groupby("Collection date").count()

,Virus name
Collection date,
2020-03,9
2020-03-06,1
2020-03-14,1
2020-03-17,2
2020-03-20,1
...,...
2022-07-23,1
2022-07-24,1
2022-07-25,3


In [708]:
#All exports
#county_cases.to_csv("cases_per_county.csv")